In [1]:
!pip install ultralytics

In [2]:
!pip install -U ipywidgets

^C
ERROR: Operation cancelled by user


In [1]:
from ultralytics import YOLO

#### Training the model

In [ ]:
results = model.train(
    data= '/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=100,
    imgsz=640,
    patience=50,
    device=0,
    batch=64,
    optimizer= 'SGD',
    momentum=0.95
)

#### Evaluation

In [ ]:
model = YOLO("best_80_10_10.pt")  

metrics = model.val()  
metrics.box.map  
metrics.box.map50  
metrics.box.map75  
metrics.box.maps  

#### hyperparameter-tuning

- in our case SGD as optimizer is better that AdamW


In [ ]:
model = YOLO("best_s_version.pt") 

- lr0 = [0.0005, **0.001**, 0.005]
- lrf = [0.01, **0.005**]
- batch = [16, **32**, 64]
- momentum = [0.85, **0.9**, 0.95]
- warmup_momentum = [**0.5**]
- warmup_bias_lr = [**0.05**, 0.1, 0.2]
- weight_decay = [0.0001, **0.0005**, 0.001]
- patience = [**30**, 50]
- imgsz = [**640**, 800, 1024]


In [ ]:
results = model.train(
    data='/kaggle/input/80-10-10-dataset/data.yaml',
    epochs=50,
    imgsz=640,
    patience=30,  
    device=0,
    batch=32,  
    optimizer='SGD',  
    lr0=0.001,  
    lrf=0.005,
    momentum=0.9,
    warmup_momentum=0.5,
    warmup_bias_lr=0.05,
    weight_decay=0.0005,  
)


#### Testing the model

In [4]:
from ultralytics.utils.plotting import Annotator
import cv2

In [3]:
model = YOLO("best_s_version.pt") 
video_path = 'sample_02.mp4'
cap = cv2.VideoCapture(video_path)

FileNotFoundError: [Errno 2] No such file or directory: 'best_s_version.pt'

In [4]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    results = model(frame)
    
    for r in results:
        annotator = Annotator(frame)
        
        boxes = r.boxes
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
    
    frame = annotator.result()
    cv2.imshow("Object Detection", frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

0: 640x384 (no detections), 78.0ms
Speed: 1.7ms preprocess, 78.0ms inference, 374.9ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 73.5ms
Speed: 1.5ms preprocess, 73.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 61.0ms
Speed: 1.2ms preprocess, 61.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 out of waters, 59.6ms
Speed: 1.3ms preprocess, 59.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 101.1ms
Speed: 2.0ms preprocess, 101.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 64.5ms
Speed: 1.5ms preprocess, 64.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 68.6ms
Speed: 1.9ms preprocess, 68.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 out of water, 60.6ms
Speed: 2.4ms preprocess, 60.6ms inference, 

#### Alarm Testing

In [ ]:
!pip install pydub pyaudio

In [7]:
from alarm import AlarmDetector
from pydub import AudioSegment
from pydub.playback import play

In [8]:
model = YOLO("../models/best_s_version.pt")
video_path = '../video_samples/sample_01.mp4'
alarm_sound = AudioSegment.from_file('alarm.wav')
cap = cv2.VideoCapture(video_path)
calc = AlarmDetector(model, 300, 50)
alarm = False


while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    if calc.check_alarm() and not alarm:
        alarm = True
        play(alarm_sound)
        play(alarm_sound)
        # add to database and send notifications
        
    results = model(frame, stream=True)
    
    for r in results:
        annotator = Annotator(frame)
        
        boxes = r.boxes
        calc.add_boxes(boxes)
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            annotator.box_label(b, model.names[int(c)])
    
    frame = annotator.result()
    cv2.imshow("Object Detection", frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 640x384 (no detections), 29.0ms
Speed: 2.0ms preprocess, 29.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 33.0ms
Speed: 2.0ms preprocess, 33.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 27.0ms
Drowning
Speed: 1.0ms preprocess, 27.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 1 Swimming, 29.0ms
Drowning
Speed: 1.0ms preprocess, 29.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 30.3ms
Drowning
Speed: 2.0ms preprocess, 30.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 Drownings, 31.0ms
Drowning
Speed: 2.0ms preprocess, 31.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 1 Swimming, 29.5ms
Drowning
Speed: 1.0ms preprocess, 29.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 Drowning, 

In [35]:
cap.release()
cv2.destroyAllWindows()

#### activate the Tracker in Yolov8